In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import matplotlib.pyplot as plt
import warnings

import pandas as pd, numpy as np, string
warnings.filterwarnings('ignore')

In [2]:
dataset = pd.read_csv('dataset/good_reads_final.csv')


In [3]:
dataset.columns

Index(['author_average_rating', 'author_gender', 'author_genres', 'author_id',
       'author_name', 'author_page_url', 'author_rating_count',
       'author_review_count', 'birthplace', 'book_average_rating',
       'book_fullurl', 'book_id', 'book_title', 'genre_1', 'genre_2',
       'num_ratings', 'num_reviews', 'pages', 'publish_date', 'score'],
      dtype='object')

In [4]:
y = dataset[['author_gender']]
dataset = dataset[['author_average_rating','author_genres', 'author_rating_count', 'author_review_count', 'birthplace', 'book_average_rating', 'book_id', 'book_title', 'genre_1', 'genre_2', 'num_ratings', 'num_reviews', 'pages', 'score']]


##### Página URL do Autor(a) foi retirado também devido a conter o nome dos Autores(as)
##### URL do Livro também foi retirada devido a trazer apenas informações que podem ser derivadas das outras *features*
##### O ano de publicação também foi retirado devido a inconsistencia da tabela (um número consideravel célula apresentavam dados inconsistentes.


In [5]:
dataset['author_average_rating'] = np.where(dataset['author_average_rating'] > 100.0, dataset['author_average_rating']/100, dataset['author_average_rating'])
dataset['book_average_rating'] = np.where(dataset['book_average_rating'] > 100.0, dataset['book_average_rating']/100, dataset['book_average_rating'])

dataset = dataset.reset_index(drop=True)
# dataset

In [6]:
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(y)
# valid_y = encoder.fit_transform(valid_y)

In [7]:
X = pd.get_dummies(dataset)

In [8]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    lista = []
    lista.append(metrics.accuracy_score(valid_y, predictions))
    lista.append(metrics.recall_score(valid_y, predictions))
    lista.append(metrics.f1_score(valid_y, predictions))
    lista.append(metrics.confusion_matrix(valid_y, predictions))
    
    return lista

In [9]:
def plotMetrics(listOfResults, listOfMetrics):

    for l in range(len(listOfResults)):
        ax = plt.axes()
        ax.bar(l, listOfResults[l], label=listOfMetrics[l], align='center', width=0.4)

    # fig = plt.figure(figsize=(1,1))
    plt.title("Análise de Gênero do Autor(a) de acordo com o titulo da obra")
    plt.legend(listOfMetrics)
    plt.ylabel("Porcentagem")
    plt.xlabel("Métricas")
    plt.xticks(range(len(listOfResults)))
    plt.show()

In [10]:
def plot_confusion_matrix(classes, cm,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Matrix de Confusão Normalizada'
        else:
            title = 'Matrix de Confusão'

    # Compute confusion matrix
    cm = cm
    # Only use the labels that appear in the data
    classes = classes
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matrix de Confusão Normalizada")
    else:
        print('Matrix de Confusão')

#     print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='Rotulo Verdadeiro',
           xlabel='Rotulo Predito')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            if i == 0:
                ax.text(j, i+0.2, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
            else:
                ax.text(j, i-0.2, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


In [11]:
models = [naive_bayes.MultinomialNB(), linear_model.LogisticRegression(), ensemble.RandomForestClassifier()]
features = [['english', None, True], ['english', 'ascii', True], 
            ['english', None, False], ['english', 'ascii', False], 
            [None, 'ascii', True], [None, 'ascii', False]]

In [13]:
d = {}
l = []
i = 0
# for md in models:
l.append(models[2])

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X,y, test_size = 0.2)

l.append(train_model(models[2], train_x, train_y, valid_x))
d[i] = l
i+=1
l = []


MemoryError: 

In [ ]:
for item in d.items():
    print(item, '\n')

### Com o print acima, podemos concluir que utilizando o método Bag of Words e os três algoritmos:

* Que o melhor cenário é não por todas as palavras em caixa baixa com remoção de caracteres não ASCII e sem remoção de stopwords;
* A remoção de StopWords, nesse caso apenas atrapalha o desempenho do algoritmo;
* Já a remoção de caracteres "não ascii" mostrou-se vantajosa;
* O algoritmo de Floresta Randômica, foi o que se saiu melhor neste experimento.

# Para TFIDF

In [ ]:
d2 = {}
l2 = []
i = 0
for md in models:
    l2.append(md)
    for f in features:
        xtrain_tfidf, xvalid_tfidf = tfidfVec(dataset['book_title'], f[0], f[1], f[2])
        l2.append(train_model(md, xtrain_tfidf, train_y, xvalid_tfidf))
    d2[i] = l2
    i+=1
    l2 = []

In [ ]:
for item in d2.items():
    print(item, '\n')

### Com o print acima, podemos concluir que utilizando o método TFIDF e os três algoritmos:

* Que o melhor cenário é não por todas as palavras em caixa baixa com remoção de caracteres não ASCII e sem remoção de stopwords;
* A remoção de StopWords, nesse caso apenas atrapalha o desempenho do algoritmo;
* Já a remoção de caracteres "não ascii" mostrou-se vantajosa;
* Mas uma vez o algoritmo de Floresta Randômica, foi o que se saiu melhor neste experimento.
* Podemos observar também que o uso da técnica TFIDF, mostrou-se obter resultados piores do que uma técnica mais simples como o Bag of Words

# Análise Preliminar dos Resultados


### Para a análise dos resultados será considerada a execução com os seguintes parâmetros:

* Sem aplicação de *lowercase*
* Com remoção de caracteres "não ASCII"
* Sem remoção de *stopwords*
* Com a aplicação da técnica *bag of words*

#### Os resultados são compostos das seguintes métricas, e seus resultados:

* Acurácia: 0,73
* Recall:   0,82
* F1 Score: 0,77

Apesar de acertar aproximadamente o gênero de 7 autores(as) a cada 10 vez, o recall do algoritmo é consideravelmente alto, o que significa que consegue identificar com uma boa margem aqueles o qual ele selecionou como sendo de cada gênero.
Todavia, observando a F1 Score, podemos inferir que a precisão do algoritmo é bem inferior ao recall, o que faz com que muitos titulos tenham sido atribuídos ao gênero errado.

In [ ]:
listOfResults = [d2[2][6][0], d2[2][6][1], d2[2][6][2]]
listOfMetrics = ['Acuracia', 'Recall', 'F1 Score']
plotMetrics(listOfResults, listOfMetrics)

In [ ]:
np.set_printoptions(precision=2)
cl = ['Homem', 'Mulher']
cl = np.asarray(cl)
plot_confusion_matrix(cl, d[2][6][3])
plot_confusion_matrix(cl, d[2][6][3], normalize=True)
plt.show()

### Com base na Matriz de Confusão, pode-se observar que o Algoritmo tem uma maior acerto (cerca de 20% a mais) quando identifica as autoras, logo ele classifica muito livro que é escrito por um autor, como sendo escrito por uma autora.